# Image Segmentation

## 1. Import Packages

In [2]:
# Import required packages
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [3]:
# check whether GPU accelerated computing is available
assert torch.cuda.is_available() # if there is an error here, enable GPU in the Runtime

## 2. Install additional Packages

In [ ]:
# install SimpleITK
!pip install SimpleITK==2.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.4 MB 1.4 MB/s 


In [ ]:
# install nnunet
!pip install nnunet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 251 kB 14.2 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 151 kB 64.3 MB/s 
     |████████████████████████████████| 61 kB 400 kB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 2.0 MB 50.8 MB/s 
     |████████████████████████████████| 13.0 MB 48.8 MB/s 
  Created wheel for nnunet: filename=nnunet-1.7.0-py3-none-any.whl size=487821 sha256=0a0f54ee5f7a8a51e45c407a8375603069a7459f3dde1c9d1ca318d50acca7f1
  Stored in directory: /root/.cache/pip/wheels/66/f6/19/c51389976a9ae30212178d1c41497893504b900f043fe1c665
  Created wheel for batchgenerators: filename=batchgenerators-0.24-py3-none-any.whl size=89209 sha256=19b2df3212700f9d299241b5e2fe8c51ac818069ce09379bb74627901a07f652
  Stored in directory: /root/.cache/pip/wheels/56/d7/ca/6b43d8fbc8e1d38a89e

In [ ]:
# install gdown to download files from GoogleDrive
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# check downloaded packages
import nnunet
import SimpleITK as sitk
import gdown

## 3. Connect Colab with Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [5]:
assert os.path.exists(drive_dir) # if this fails, something went wrong with mounting GoogleDrive
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)

## 4. Set up nnU-Nets folder structure and environment variables

In [6]:
# define helper function to create folder structure in Colab
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [7]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw_data_base" : os.path.join(mount_dir, "nnUNet_raw_data_base"), 
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), 
    "RESULTS_FOLDER" : os.path.join(mount_dir, "nnUNet_Results_Folder"),
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward.")

Current Working Directory /content
/content/drive/My Drive/Colab Notebooks/nnUNet_raw_data_base exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder exists.
If No Error Occured Continue Forward.


## 5. Download the Data

In [ ]:
# set path to raw MRI images
raw_data_dir = os.path.join(mount_dir, "nnUNet_raw_data_base/nnUNet_raw_data")

# check if MRI images are already downloaded
if os.path.exists(os.path.join(raw_data_dir, "T2_test_data")):
  print("Images are already downloaded. Please do not excute the following code line!")
else:
  os.chdir(raw_data_dir)
  print("Images are not yet downloaded. Please proceed to the following code line.")

In [ ]:
# NOTE! only execute this command line if the images are not yet downloaded
os.chdir(raw_data_dir)

# download images from Drive
!gdown --fuzzy https://drive.google.com/file/d/14W4XAt_opWVGdx0tPZDMnXwvNEktzxiy/view?usp=share_link
# unzip the images
!unzip T2_test_data.zip
# set path to new directory
image_dir = os.path.join(raw_data_dir, "T2_test_data")

Downloading...
From: https://drive.google.com/uc?id=14W4XAt_opWVGdx0tPZDMnXwvNEktzxiy
To: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw_data_base/nnUNet_raw_data/T2_test_data.zip
100% 152M/152M [00:00<00:00, 238MB/s]


## 6. Create Metadata File of the Data

In [ ]:
# specify helper function to create a dataset.json file
from typing import Tuple
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *


def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-12] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques


def generate_dataset_json(output_file: str, imagesTr_dir: str, imagesTs_dir: str, modalities: Tuple,
                          labels: dict, dataset_name: str, sort_keys=True, license: str = "hands off!", dataset_description: str = "",
                          dataset_reference="", dataset_release='0.0'):
  
    train_identifiers = get_identifiers_from_splitted_files(imagesTr_dir)

    if imagesTs_dir is not None:
        test_identifiers = get_identifiers_from_splitted_files(imagesTs_dir)
    else:
        test_identifiers = []

    json_dict = {}
    json_dict['name'] = dataset_name
    json_dict['description'] = dataset_description
    json_dict['tensorImageSize'] = "4D"
    json_dict['reference'] = dataset_reference
    json_dict['licence'] = license
    json_dict['release'] = dataset_release
    json_dict['modality'] = {str(i): modalities[i] for i in range(len(modalities))}
    json_dict['labels'] = {str(i): labels[i] for i in labels.keys()}

    json_dict['numTraining'] = len(train_identifiers)
    json_dict['numTest'] = len(test_identifiers)
    json_dict['training'] = [
        {'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i
        in
        train_identifiers]
    json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_identifiers]

    if not output_file.endswith("dataset.json"):
        print("WARNING: output file name is not dataset.json! This may be intentional or not. You decide. "
              "Proceeding anyways...")
    save_json(json_dict, os.path.join(output_file), sort_keys=sort_keys)

In [ ]:
# generate dataset.json file for T2 weighted images using helper function
generate_dataset_json(output_file=os.path.join(raw_data_dir, 'T2_test_data/dataset.json'), 
                      imagesTr_dir=os.path.join(raw_data_dir, 'T2_test_data/imagesTr'), 
                      imagesTs_dir=os.path.join(raw_data_dir, 'T2_test_data/imagesTs'),
                      modalities=("T2"),
                      labels={0: 'background', 
                              1: 'left kidney', 
                              2: 'right kidney',
                              3: 'liver',
                              4: 'spleen'}, 
                      dataset_name='Chaos segmentation challenge', 
                      sort_keys=True, 
                      license='NA', 
                      dataset_description='Abdominal MRI scans of healthy individuals',
                      dataset_reference='NA', 
                      dataset_release='1.1.1111')

## 7. Inference on Test Data using Pretrained Model

### 7.1 Download the Pretrained Model

In [ ]:
# Download the Pretrained Liver Segmentation Model
!nnUNet_download_pretrained_model Task038_CHAOS_Task_3_5_Variant2



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet


######################################################
!!!!!!!!!!!!!!!!!!!!!!!!WARNING!!!!!!!!!!!!!!!!!!!!!!!
######################################################
Using the pretrained model weights is subject to the license of the dataset they were trained on. Some allow commercial use, others don't. It is your responsibility to make sure you use them appropriately! Use nnUNet_print_pretrained_model_info(task_name) to see a summary of the dataset and where to find its license!
######################################################

Download finished. Extracting...
Traceback (most recent call last):
  File "/usr/local/bin/nnUNet

In [ ]:
# obtain information about the model
!nnUNet_print_pretrained_model_info Task038_CHAOS_Task_3_5_Variant2



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

CHAOS - Combined (CT-MR) Healthy Abdominal Organ Segmentation Challenge (Task 3 & 5). 
Segmentation targets are left and right kidney, liver, spleen, 
Input modalities are 0: T1 in-phase, T1 out-phase, T2 (can be any of those)
Also see https://chaos.grand-challenge.org/


### 7.2 Inference on the Test Data

In [ ]:
# use fully trained nnU-Net to make predictions on data (2d)
# store results in predTs folder
!nnUNet_predict -i "${nnUNet_raw_data_base}/nnUNet_raw_data/T2_test_data/imagesTs" -o "${RESULTS_FOLDER}/T2_test_data/predTs" -t Task038_CHAOS_Task_3_5_Variant2 -m 2d



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder/nnUNet/2d/Task038_CHAOS_Task_3_5_Variant2/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 40 unique case ids, here are some examples: ['LIVER_011' 'LIVER_037' 'LIVER_005' 'LIVER_019' 'LIVER_032' 'LIVER_025'
 'LIVER_015' 'LIVER_036' 'LIVER_014' 'LIVER_007']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 40
number of cases that still need to be predicted: 40
emptying cuda cache
loading parameters for folds, None
folds is None so we w